# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [77]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [78]:

print(os.getcwd())


filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):

    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [79]:

full_data_rows_list = [] 

for f in file_path_list:

 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 

        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
     
        for line in csvreader:

            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [80]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [81]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [82]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [83]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



    





In [84]:
query = ("""

        CREATE TABLE IF NOT EXISTS songs_session
        (sessionId int, itemInSession int, artist text, song_title text, song_length float,
        PRIMARY KEY(sessionId, itemInSession))

        """)                    
session.execute(query)

In [85]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [86]:
#For table songs_session, the column sessionId and itemInSesssion were used as a partition key because the queries will filter by these columns.
select_data = session.execute("""SELECT artist, song_title, song_length FROM songs_session WHERE itemInSession = 4 AND sessionId = 338""")
for row in select_data:
    print("Artist, Name, and Length: ", row.artist, "/", row.song_title, "/", row.song_length)

Artist, Name, and Length:  Faithless / Music Matters (Mark Knight Dub) / 495.30731201171875



### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [87]:
query = ("""
    
    CREATE TABLE IF NOT EXISTS art_song_user
        (user_id int,
         session_id int,
         item_in_session int,
         artist text,
         song text,
         PRIMARY KEY ((user_id, session_id), item_in_session))
    
        """)                    
session.execute(query)                    

In [88]:
# Insert Data for Query 2:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO art_song_user (user_id, session_id, item_in_session, artist, song, first_name, last_name, )"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [89]:
# Query the Data for Query 2
# For table art_song_user, the columns user_id, and session_id were used as partition keys because the queries will filter by these columns. item_in_session is used as a clustering column to help make up a unique key.
query = "SELECT artist, song, first_name, last_name FROM art_song_user WHERE user_id=10 AND session_id=182"
select_data = session.execute(query)
for row in select_data:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

                    

In [90]:
query = ("""
    
    CREATE TABLE IF NOT EXISTS users_list
        (
         song text,
         user_id int,
         first_name text,
         last_name text,

         PRIMARY KEY (song, user_id))
    
        """)                    
session.execute(query)   

In [91]:
# Insert Data for Query 3:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO users_list (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [92]:
# Query the Data for Query 3
# For table art_song_user, the columns song, and user_id were used as partition keys because the queries will filter by these columns.
query = "SELECT first_name, last_name FROM users_list WHERE song='All Hands Against His Own'"
select_data = session.execute(query)
for row in select_data:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [93]:
session.execute("drop table songs_session")
session.execute("drop table art_song_user")
session.execute("drop table users_list")

### Close the session and cluster connection¶

In [94]:
session.shutdown()
cluster.shutdown()

### Thanks All, hope you enjoyed, and a big thanks to Udacity for the great content!